In [2]:
from dask.distributed import Client
import dask.dataframe as dd
import pandas as pd
import numpy as np

OUTPUT_DATA_PATH = "../../data/sample/"

In [3]:
client = Client("tcp://localhost:8786")
client

/home/nicolas/GDrive/educacao/00-pos/UNB/20250328_PPCA0027_MDM_T01/01-ppca_mdm_trabalho/ppca_mdm_pgfn_prev_jud_dask/venv/lib/python3.12/site-packages/distributed/client.py:1579: VersionMismatchWarning: Mismatched versions found

+---------+----------------+-----------------+-----------------+
| Package | Client         | Scheduler       | Workers         |
+---------+----------------+-----------------+-----------------+
| python  | 3.12.3.final.0 | 3.10.12.final.0 | 3.10.12.final.0 |
+---------+----------------+-----------------+-----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://172.23.0.3:8786' processes=2 threads=32, memory=61.92 GiB>

In [6]:

# ✅ Criar CSVs de exemplo (pule se já tiver dados)
df1 = pd.DataFrame({
    "id": range(1, 100001),
    "categoria": np.random.choice(["A", "B", "C", "D"], size=100000),
    "valor": np.random.rand(100000) * 1000
})
df1.to_csv(f"{OUTPUT_DATA_PATH}/data1.csv", index=False)

df2 = pd.DataFrame({
    "id": range(50000, 150000),
    "descricao": np.random.choice(["foo", "bar", "baz"], size=100000)
})
df2.to_csv(f"{OUTPUT_DATA_PATH}/data2.csv", index=False)

# ✅ Leitura com Dask
d1 = dd.read_csv(f"{OUTPUT_DATA_PATH}/data1.csv")
d2 = dd.read_csv(f"{OUTPUT_DATA_PATH}/data2.csv")


# ✅ Operações básicasd'1
print("Head d1:")
print(d1.head())

# 🎯 Filtro avançado
filtro = d1[(d1.valor > 900) & (d1.categoria.isin(["B", "C"]))]
print("Filtro avançado:\n", filtro.head())

# 📈 Média por categoria
print("Média por categoria:")
print(d1.groupby("categoria").valor.mean().compute())

# 🔗 Merge (join) entre DataFrames
merged = d1.merge(d2, on="id", how="inner")
print("Merge feito!")
print(merged.head())

# ✅ Salvar como Parquet particionado por categoria
merged.to_parquet(f"{OUTPUT_DATA_PATH}saida_parquet", partition_on=["categoria"], engine="pyarrow", write_index=False)

# ✅ Ler novamente do Parquet
df_parquet = dd.read_parquet(f"{OUTPUT_DATA_PATH}saida_parquet", engine="pyarrow")
print("Leitura do Parquet particionado:\n", df_parquet.head())

# ✅ Ver o cluster no dashboard web
print("Dashboard link:", client.dashboard_link)



Head d1:


FileNotFoundError: [Errno 2] No such file or directory: '/home/nicolas/GDrive/educacao/00-pos/UNB/20250328_PPCA0027_MDM_T01/01-ppca_mdm_trabalho/ppca_mdm_pgfn_prev_jud_dask/notebooks/examples/../../data/sample//data1.csv'

In [ ]:
d1

In [ ]:
d1

In [8]:
import dask.dataframe as dd

# Definir as credenciais programaticamente
boto3.client('s3', aws_access_key_id='sua_access_key',
                  aws_secret_access_key='sua_secret_key', 
                  region_name='sua_regiao')

# Ou você pode configurar o boto3 no próprio código:
config.set({"aws.access_key_id": "minioadmin", 
            "aws.secret_access_key": "minioadmin123", 
            "region": "us-east-1"})

df = dd.read_parquet("s3://ppca/cnoj/raw/cnpj.parquet", engine="pyarrow")

# How much did NYC pay Uber?
df.cd_cnpj.count().compute()

# And how much did drivers make?
df.cd_cnpj.count().compute()

NameError: name 'boto3' is not defined

In [ ]:
from dask.distributed import LocalCluster
client = LocalCluster().get_client()